In [1]:
import os
import glob

import time
import datetime

import math

import numpy as np
import pandas as pd

from matplotlib import pyplot as plt

In [2]:
def rot_x(phi):
    rot_mat = np.matrix([
        [1, 0, 0],
        [0, math.cos(phi), math.sin(phi)],
        [0, -math.sin(phi), math.cos(phi)],
    ]
    )
    
    for i in range(3):
        for j in range(3):
            if abs(rot_mat[i, j]) < 10 ** -6:
                rot_mat[i, j] = 0
    
    return rot_mat

def rot_y(theta):
    rot_mat = np.matrix([
        [math.cos(theta), 0, -math.sin(theta)],
        [0, 1, 0],
        [math.sin(theta), 0, math.cos(theta)],
    ]
    )

    for i in range(3):
        for j in range(3):
            if abs(rot_mat[i, j]) < 10 ** -6:
                rot_mat[i, j] = 0
    
    return rot_mat

def rot_z(psi):
    rot_mat = np.matrix([
        [math.cos(psi), math.sin(psi), 0],
        [-math.sin(psi), math.cos(psi), 0],
        [0, 0, 1],
    ]
    )

    for i in range(3):
        for j in range(3):
            if abs(rot_mat[i, j]) < 10 ** -6:
                rot_mat[i, j] = 0
    
    return rot_mat

def mat_str(m):
    s = ''
    
    for i in range(3):
        for j in range(3):
            d = int(m[i, j])
            
            if abs(d - m[i,j])  > 10 ** -6:
                print('float value!', d, m[i,j], abs(d - m[i,j]))
            
            s += str(d) + ' '
            
    return s

def mat_dict_update(x, y, z, mat_dict):
    x_mat = x_list[x]
    y_mat = y_list[y]
    z_mat = z_list[z]
    
    x_l = 'x{}'.format(x)    
    y_l = 'y{}'.format(y)    
    z_l = 'z{}'.format(z)    
    
    c_list = [
        (x_l + y_l + z_l, x_mat * y_mat * z_mat),
        (x_l + z_l + y_l, x_mat * z_mat * y_mat),
        (y_l + x_l + z_l, y_mat * x_mat * z_mat),
        (y_l + z_l + x_l, y_mat * z_mat * x_mat),
        (z_l + x_l + y_l, z_mat * x_mat * y_mat),
        (z_l + y_l + x_l, z_mat * y_mat * x_mat),
    ]
    
    for l, m in c_list:
        m_s = mat_str(m)
        
        if m_s not in mat_dict.keys():
            mat_dict[m_s] = []
        
        mat_dict[m_s].append(l)
    
    return mat_dict

In [3]:
x_list = [rot_x(i * math.pi/2) for i in range(4)]
y_list = [rot_y(i * math.pi/2) for i in range(4)]
z_list = [rot_z(i * math.pi/2) for i in range(4)]

mat_dict = {}

for x in range(4):
    for y in range(4):
        for z in range(4):
            mat_dict = mat_dict_update(x, y, z, mat_dict)

mat_list = []

for k in mat_dict.keys():
    vals = list(map(int, k.split()))
    vals = np.array(vals).reshape((3, 3))

    m = np.matrix(vals)
    mat_list.append(m)    

In [4]:
def rotate(ref_pos, pos, mat):
    rot_pos = []
    
    for i in range(3):
        v = 0
        
        for j in range(3):
            v += mat[i, j] * (pos[j] - ref_pos[j])
            
        v += ref_pos[i]
        rot_pos.append(v)
        
    return rot_pos

In [5]:
def block_same_check(block_1, block_2):
    s_block_1 = sorted(block_1)
    s_block_2 = sorted(block_2)
    
    ref_1 = s_block_1[0]
    ref_2 = s_block_2[0]
    
    for i in range(len(block_1)):
        p_1 = list(s_block_1[i])
        p_2 = list(s_block_2[i])
        
        for j in range(3):
            p_1[j] -= ref_1[j]
            p_2[j] -= ref_2[j]
            
            if p_1[j] != p_2[j]:
                return False
    return True

In [6]:
def get_rotation_index(pos_list_1, pos_list_2):
    ref_pos_1 = pos_list_1[0]
    rotation_list = []
    
    for rotation in range(24):
        rot_pos_list_1 = []
        
        for pos_1 in pos_list_1:
            rot_pos = rotate(ref_pos_1, pos_1, mat_list[rotation])
            rot_pos_list_1.append(rot_pos)
            
        same = block_same_check(rot_pos_list_1, pos_list_2)
        
        if same:
            rotation_list.append(rotation)
    
    return rotation_list

In [7]:
CHAMP_PATH = os.path.join('/home', 'jovyan', 'work', 'result', 'champion', 'champ_all.csv')
champ_df = pd.read_csv(CHAMP_PATH)

In [31]:
seed = 1401
result = champ_df.query('seed == {}'.format(seed))['champion_result'].to_list()[0]
D = int(champ_df.query('seed == {}'.format(seed))['D'].to_list()[0])

In [32]:
champ_df.query('seed == {}'.format(seed))

,seed,D,problem,champion,champion_score,champion_result
1269,1401,14,14 11111110000000 11111110000000 1111111000000...,cellpair-3-beam-5-50K,7254347,2R0-2-2-2-2-2-2-2-2-2-2-2-2-2-0-2-2-2-2-2-2-2-...


In [2]:
# 手動設定
result = '2R0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-1-1-1-2-2-0-1-1-1-2-0-0-0-1-2-0-0-0-1-0-0-0-0-0-0-1-1-1-2-2-0-1-1-2-2-0-0-0-2-2-0-0-0-2-0-0-0-0-0-0R0-2-2-2-2-0-1-1-1-2-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-1-1-2-2-2-1-1-2-2-2-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-1-1-0-0-0-1-1-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-1-0-0-0-0-1-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0'

In [3]:
print(result.replace('R', '\n').replace('-', ' '))

2
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 2 2 0 1 1 1 2 0 0 0 1 2 0 0 0 1 0 0 0 0 0 0 1 1 1 2 2 0 1 1 2 2 0 0 0 2 2 0 0 0 2 0 0 0 0 0 0
0 2 2 2 2 0 1 1 1 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 2 2 2 1 1 2 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0


In [30]:
print(result.count('1'))
print(result.count('2'))

38
25


In [20]:
shape_1 = list(map(int, result.split('R')[1].split('-')))
shape_2 = list(map(int, result.split('R')[2].split('-')))

block_1_1 = []
block_1_2 = []
block_2_1 = []
block_2_2 = []

for z in range(D):
    for y in range(D):
        for x in range(D):
            index = D**2 * x + D * y + z
            
            block_no = shape_1[index]
            
            if block_no == 1:
                block_1_1.append((x, y, z))
            elif block_no == 2:
                block_1_2.append((x, y, z))

            block_no = shape_2[index]
            
            if block_no == 1:
                block_2_1.append((x, y, z))
            elif block_no == 2:
                block_2_2.append((x, y, z))


In [18]:
def set_block(index, block, block_no):
    for x, y, z in block:
        print('shapes_[{}].SetBlockNo({{{}, {}, {}}}, {});'.format(index, x, y, z, block_no))

In [19]:
def block_1_hash(block):
    x, y, z = block[0]
    
    print("Block debug_block(D_, {{{}, {}, {}}});".format(x, y, z))

    for x, y, z in block[1:]:
        print('debug_block.AddBlock({{{}, {}, {}}});'.format(x, y, z))
        
    print('debug(debug_block.Hash(0));')

In [20]:
get_rotation_index(block_1_2, block_2_2)

[20]

In [21]:
len(block_1_1)

10

In [22]:
len(block_1_2)

12

In [23]:
block_1_1

[(0, 2, 0),
 (0, 1, 1),
 (0, 2, 1),
 (0, 3, 1),
 (0, 2, 2),
 (1, 2, 2),
 (0, 3, 2),
 (1, 3, 2),
 (0, 1, 3),
 (0, 2, 3)]

In [24]:
block_1_2

[(0, 0, 3),
 (1, 0, 3),
 (2, 0, 3),
 (1, 1, 3),
 (2, 1, 3),
 (1, 2, 3),
 (2, 2, 3),
 (0, 1, 4),
 (1, 1, 4),
 (2, 1, 4),
 (1, 2, 4),
 (2, 2, 4)]

In [216]:
block_2_1

[(4, 2, 0),
 (4, 3, 0),
 (4, 3, 1),
 (4, 3, 2),
 (3, 4, 2),
 (4, 4, 2),
 (3, 4, 3),
 (4, 4, 3),
 (3, 4, 4),
 (4, 4, 4)]

In [217]:
block_2_2

[(3, 0, 0),
 (2, 1, 0),
 (3, 1, 0),
 (4, 1, 0),
 (2, 1, 1),
 (4, 1, 1),
 (2, 2, 1),
 (2, 3, 1),
 (3, 3, 1),
 (2, 4, 1),
 (3, 4, 1),
 (4, 4, 1),
 (3, 3, 2)]

In [218]:
set_block(0, block_1_1, 2)

shapes_[0].SetBlockNo({1, 2, 0}, 2);
shapes_[0].SetBlockNo({1, 3, 0}, 2);
shapes_[0].SetBlockNo({1, 3, 1}, 2);
shapes_[0].SetBlockNo({1, 3, 2}, 2);
shapes_[0].SetBlockNo({0, 4, 2}, 2);
shapes_[0].SetBlockNo({1, 4, 2}, 2);
shapes_[0].SetBlockNo({0, 4, 3}, 2);
shapes_[0].SetBlockNo({1, 4, 3}, 2);
shapes_[0].SetBlockNo({0, 4, 4}, 2);
shapes_[0].SetBlockNo({1, 4, 4}, 2);


In [219]:
set_block(1, block_2_1, 2)

shapes_[1].SetBlockNo({4, 2, 0}, 2);
shapes_[1].SetBlockNo({4, 3, 0}, 2);
shapes_[1].SetBlockNo({4, 3, 1}, 2);
shapes_[1].SetBlockNo({4, 3, 2}, 2);
shapes_[1].SetBlockNo({3, 4, 2}, 2);
shapes_[1].SetBlockNo({4, 4, 2}, 2);
shapes_[1].SetBlockNo({3, 4, 3}, 2);
shapes_[1].SetBlockNo({4, 4, 3}, 2);
shapes_[1].SetBlockNo({3, 4, 4}, 2);
shapes_[1].SetBlockNo({4, 4, 4}, 2);


In [23]:
block_1_hash(block_1_2)

Block debug_block(D_, {2, 1, 0});
debug_block.AddBlock({1, 1, 1});
debug_block.AddBlock({2, 1, 1});
debug_block.AddBlock({1, 1, 2});
debug_block.AddBlock({2, 1, 2});
debug_block.AddBlock({3, 1, 2});
debug_block.AddBlock({2, 1, 3});
debug_block.AddBlock({3, 1, 3});
debug_block.AddBlock({2, 1, 4});
debug_block.AddBlock({3, 1, 4});
debug_block.AddBlock({2, 2, 4});
debug_block.AddBlock({3, 2, 4});
debug(debug_block.Hash(0));
